In [1]:
import pandas as pd
import os
import json
import xlrd
import datetime 
import locale
import numpy as np
from openpyxl import load_workbook

In [2]:
# ganti ini ya, sesuai dengan kebutuhan kalian
GLOBAL_VAR = {
    "kode_satker":"1507",
    "master_sls_pusat":"Master SLS tanjabbar 2023.xlsx",
    "sheet_name_pusat":"Sheet1",
    "master_sls_kab":"Petugas ST Per Kecamatan.xlsx",
    "sheet_name_kab":"Kabupaten",
    "output_dir":"hasil",
    "output_filename": 'Pemutakhiran SLS.xlsx'
}

In [51]:
folder = "data"
masterSLSPusat = pd.read_excel(folder+"/"+GLOBAL_VAR["master_sls_pusat"], sheet_name=GLOBAL_VAR["sheet_name_pusat"], converters={
        "IDSLS":str,
        "KODE PROVINSI":str,
        "KODE KECAMATAN":str,
        "KODE DESA":str,
        "KODE SLS":str,
        "KODE KABUPATEN":str

})
masterSLSKab = pd.read_excel(folder+"/"+GLOBAL_VAR["master_sls_kab"], sheet_name=GLOBAL_VAR["sheet_name_kab"], converters={
    "idsls":str
})

In [4]:
masterSLSPusat.head()

,KODE PROVINSI,PROVINSI,KODE KABUPATEN,KABUPATEN,KODE KECAMATAN,KECAMATAN,KODE DESA,DESA,KLASIFIKASI DESA KELURAHAN,KODE SLS,NAMA SLS,PERKIRAAN JUMLAH KK,PERKIRAAN JUMLAH KELUARGA PERTANIAN,PERKIRAAN JUMLAH KELUARGA HASIL REGSOSEK,KLASIFIKASI KONSENTRASI PERTANIAN,METODE PENDATAAN,SUMBER,IDSLS
0,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,000100,RT 01 DUSUN 1,61,48,62,Konsentrasi,Door to door,Wilkerstat,1507010016000100
1,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,000200,RT 04 DUSUN 1,54,47,64,Konsentrasi,Door to door,Wilkerstat,1507010016000200
2,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,000300,RT 02 DUSUN 2,46,39,43,Konsentrasi,Door to door,Wilkerstat,1507010016000300
3,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,000400,RT 03 DUSUN 2,35,25,38,Konsentrasi,Door to door,Wilkerstat,1507010016000400
4,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,000500,RT 05 DUSUN 2,29,14,48,Non Konsentrasi,Snowball,Wilkerstat,1507010016000500


In [52]:
masterSLSKab.head()

,provinsi,kabupaten,kecamatan,desa,nama sls,idsls,Jumlah Keluarga REGSOSEK,Nomor Urut Bangunan Terbesar REGSOSEK,"KONSENTRASI (1=KONSENTRASI, 0=NONKONSENTRASI)",Jumlah KK PEMETAAN,Jumlah KK Pertanian,PLS,PML,Koseka
0,[15] JAMBI,[1507] TANJUNG JABUNG BARAT,[1507010] TUNGKAL ULU,[1507010016] BADANG,RT 01 DUSUN 1,1507010016000100,62,49,1,61,48,0,0,Toharun
1,[15] JAMBI,[1507] TANJUNG JABUNG BARAT,[1507010] TUNGKAL ULU,[1507010016] BADANG,RT 04 DUSUN 1,1507010016000200,64,44,1,54,47,0,0,Toharun
2,[15] JAMBI,[1507] TANJUNG JABUNG BARAT,[1507010] TUNGKAL ULU,[1507010016] BADANG,RT 02 DUSUN 2,1507010016000300,43,30,1,46,39,0,0,Toharun
3,[15] JAMBI,[1507] TANJUNG JABUNG BARAT,[1507010] TUNGKAL ULU,[1507010016] BADANG,RT 03 DUSUN 2,1507010016000400,38,30,1,35,25,0,0,Toharun
4,[15] JAMBI,[1507] TANJUNG JABUNG BARAT,[1507010] TUNGKAL ULU,[1507010016] BADANG,RT 05 DUSUN 2,1507010016000500,48,42,0,29,14,0,0,Toharun


In [53]:
masterSLSKab['kode provinsi'] = masterSLSKab.apply(lambda row : row.provinsi[1:3], axis=1)
masterSLSKab['kode kabupaten'] = masterSLSKab.apply(lambda row : row.kabupaten[3:5], axis=1)
masterSLSKab['kode kecamatan'] = masterSLSKab.apply(lambda row : row.kecamatan[5:8], axis=1)
masterSLSKab['kode desa'] = masterSLSKab.apply(lambda row : row.desa[8:11], axis=1)
masterSLSKab['kode sls'] = masterSLSKab.apply(lambda row : row.idsls[-6:], axis=1)

masterSLSKab['provinsi'] = masterSLSKab.apply(lambda row : row.provinsi[4:].strip(), axis=1)
masterSLSKab['kabupaten'] = masterSLSKab.apply(lambda row : row.kabupaten[6:].strip(), axis=1)
masterSLSKab['kecamatan'] = masterSLSKab.apply(lambda row : row.kecamatan[9:].strip(), axis=1)
masterSLSKab['desa'] = masterSLSKab.apply(lambda row : row.desa[12:].strip(), axis=1)


In [54]:
masterSLSKab.head()

,provinsi,kabupaten,kecamatan,desa,nama sls,idsls,Jumlah Keluarga REGSOSEK,Nomor Urut Bangunan Terbesar REGSOSEK,"KONSENTRASI (1=KONSENTRASI, 0=NONKONSENTRASI)",Jumlah KK PEMETAAN,Jumlah KK Pertanian,PLS,PML,Koseka,kode provinsi,kode kabupaten,kode kecamatan,kode desa,kode sls
0,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 01 DUSUN 1,1507010016000100,62,49,1,61,48,0,0,Toharun,15,07,010,016,000100
1,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 04 DUSUN 1,1507010016000200,64,44,1,54,47,0,0,Toharun,15,07,010,016,000200
2,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 02 DUSUN 2,1507010016000300,43,30,1,46,39,0,0,Toharun,15,07,010,016,000300
3,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 03 DUSUN 2,1507010016000400,38,30,1,35,25,0,0,Toharun,15,07,010,016,000400
4,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 05 DUSUN 2,1507010016000500,48,42,0,29,14,0,0,Toharun,15,07,010,016,000500


In [9]:
idslsPusatUnique = masterSLSPusat.IDSLS.unique()
idslsPusat=np.array(idslsPusatUnique).tolist()
print('banyak id sls pusat', len(idslsPusat))

idslsKabUnique = masterSLSKab.idsls.unique()
idslsKab = np.array(idslsKabUnique).tolist()
print('banyak id sls kab', len(idslsKab))

allSls = idslsPusat + idslsKab
idslss = list(set(allSls))
print("total sls ", len(idslss))

banyak id sls pusat 1805
banyak id sls kab 1805
total sls  1813


In [17]:
# yang beririsan
idslsKabNPusat = [x for x in idslsPusat if x in idslsKab]

# ada di pusat, ngga ada di kabupaten
idslsPusatNotKab = [x for x in idslsPusat if x not in idslsKab]

# ada di kabupaten, ngga ada di pusat
idslsKabNotPusat = [x for x in idslsKab if x not in idslsPusat]

print('ada di pusat dan kabupaten :', len(idslsKabNPusat), '->', idslsKabNPusat)
print('ada di pusat ngga ada di kabupaten :', len(idslsPusatNotKab), '->', idslsPusatNotKab)
print('ada di kabupaten ngga ada di pusat : ', len(idslsKabNotPusat), '->', idslsKabNotPusat)
print('total sls unik :', len(idslsKabNPusat)+len(idslsPusatNotKab)+len(idslsKabNotPusat))

ada di pusat dan kabupaten : 1797 -> ['1507010016000100', '1507010016000200', '1507010016000300', '1507010016000400', '1507010016000500', '1507010016100100', '1507010016100200', '1507010017000100', '1507010017000200', '1507010017000300', '1507010017000400', '1507010017000500', '1507010017000600', '1507010017000700', '1507010017000800', '1507010017000900', '1507010017100100', '1507010017100200', '1507010017100300', '1507010017100400', '1507010017100500', '1507010018000100', '1507010018000200', '1507010018000300', '1507010018000400', '1507010018000500', '1507010018000600', '1507010018000700', '1507010018000800', '1507010018000900', '1507010018001000', '1507010018001100', '1507010018100100', '1507010018100200', '1507010018100300', '1507010018100400', '1507010019000100', '1507010019000200', '1507010019000300', '1507010019000400', '1507010019000500', '1507010019000600', '1507010019000700', '1507010019000800', '1507010019100100', '1507010025000100', '1507010025000200', '1507010025000300', '1

In [20]:

# cek keunikan
print(len(set(idslsKabNPusat)))
print(len(set(idslsPusatNotKab)))
print(len(set(idslsKabNotPusat)))

1797
8
8


In [36]:
masterSLSKab[masterSLSKab['idsls']=='1507012001002500']

,provinsi,kabupaten,kecamatan,desa,nama sls,idsls,Jumlah Keluarga REGSOSEK,Nomor Urut Bangunan Terbesar REGSOSEK,"KONSENTRASI (1=KONSENTRASI, 0=NONKONSENTRASI)",Jumlah KK PEMETAAN,Jumlah KK Pertanian,PLS,PML,Koseka,kode provinsi,kode kabupaten,kode kecamatan,kode desa,kode sls


In [70]:

kolomOutput = [
    "idsls",
    "kode provinsi",
    "provinsi",
    "kode kabupaten",
    "kabupaten",
    "kode kecamatan",
    "kecamatan",
    "kode desa",
    "desa",
    "kode sls",
    "nama sls",
    ]

In [63]:
# add rule
newCol = [x.upper() for x in kolomOutput]
slsPusatNKab = masterSLSPusat[masterSLSPusat['IDSLS'].isin(idslsKabNPusat)][newCol]
slsPusatNKab.columns = kolomOutput
slsPusatNKab['ada di master sls pusat'] = 1
slsPusatNKab['ada di master sls kabupaten']=1

slsPusatNotKab = masterSLSPusat[masterSLSPusat['IDSLS'].isin(idslsPusatNotKab)][newCol]
slsPusatNotKab.columns = kolomOutput
slsPusatNotKab['ada di master sls pusat'] = 1
slsPusatNotKab['ada di master sls kabupaten']=0

slsKabNotPusat = masterSLSKab[masterSLSKab['idsls'].isin(idslsKabNotPusat)][kolomOutput]
slsKabNotPusat['ada di master sls pusat'] = 0
slsKabNotPusat['ada di master sls kabupaten']=1


In [64]:
slsPusatNKab.head()

,idsls,kode provinsi,provinsi,kode kabupaten,kabupaten,kode kecamatan,kecamatan,kode desa,desa,kode sls,nama sls,ada di master sls pusat,ada di master sls kabupaten
0,1507010016000100,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,000100,RT 01 DUSUN 1,1,1
1,1507010016000200,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,000200,RT 04 DUSUN 1,1,1
2,1507010016000300,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,000300,RT 02 DUSUN 2,1,1
3,1507010016000400,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,000400,RT 03 DUSUN 2,1,1
4,1507010016000500,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,000500,RT 05 DUSUN 2,1,1


In [65]:
slsPusatNotKab.head()

,idsls,kode provinsi,provinsi,kode kabupaten,kabupaten,kode kecamatan,kecamatan,kode desa,desa,kode sls,nama sls,ada di master sls pusat,ada di master sls kabupaten
262,1507012001002500,15,JAMBI,07,TANJUNG JABUNG BARAT,012,BATANG ASAM,001,LUBUK BERNAI,002500,RT 025,1,0
263,1507012001002600,15,JAMBI,07,TANJUNG JABUNG BARAT,012,BATANG ASAM,001,LUBUK BERNAI,002600,RT 026,1,0
988,1507020014001000,15,JAMBI,07,TANJUNG JABUNG BARAT,020,PENGABUAN,014,SUNGAI JERING,001000,RT. 010 DUSUN TANJUNG BARU,1,0
1342,1507030016001700,15,JAMBI,07,TANJUNG JABUNG BARAT,030,TUNGKAL ILIR,016,SRIWIJAYA,001700,RT 015,1,0
1395,1507031001001601,15,JAMBI,07,TANJUNG JABUNG BARAT,031,BRAM ITAM,001,BRAM ITAM KANAN,001601,RT 16 DUSUN SUKA MAKMUR,1,0


In [66]:
slsKabNotPusat

,idsls,kode provinsi,provinsi,kode kabupaten,kabupaten,kode kecamatan,kecamatan,kode desa,desa,kode sls,nama sls,ada di master sls pusat,ada di master sls kabupaten
268,1507012001900100,15,JAMBI,07,TANJUNG JABUNG BARAT,012,BATANG ASAM,001,LUBUK BERNAI,900100,RT 025,0,1
269,1507012001900200,15,JAMBI,07,TANJUNG JABUNG BARAT,012,BATANG ASAM,001,LUBUK BERNAI,900200,RT 026,0,1
988,1507020014900100,15,JAMBI,07,TANJUNG JABUNG BARAT,020,PENGABUAN,014,SUNGAI JERING,900100,RT. 010 DUSUN TANJUNG BARU,0,1
1342,1507030016900100,15,JAMBI,07,TANJUNG JABUNG BARAT,030,TUNGKAL ILIR,016,SRIWIJAYA,900100,RT 015,0,1
1395,1507031001001600,15,JAMBI,07,TANJUNG JABUNG BARAT,031,BRAM ITAM,001,BRAM ITAM KANAN,001600,RT 15 DUSUN SUKA MAKMUR,0,1
1396,1507031001001700,15,JAMBI,07,TANJUNG JABUNG BARAT,031,BRAM ITAM,001,BRAM ITAM KANAN,001700,RT 16 DUSUN SUKA MAKMUR,0,1
1470,1507031008001700,15,JAMBI,07,TANJUNG JABUNG BARAT,031,BRAM ITAM,008,BRAM ITAM RAYA,001700,RT 010 DUSUN BUMI AYU,0,1
1471,1507031008001800,15,JAMBI,07,TANJUNG JABUNG BARAT,031,BRAM ITAM,008,BRAM ITAM RAYA,001800,RT 011 DUSUN BUMI AYU,0,1


In [123]:
# merge tiga df
slsFinal = slsPusatNKab.append(slsPusatNotKab).append(slsKabNotPusat)


In [128]:
slsFinal['nama berubah'] = 0
slsFinal['nama di sls kab'] = slsFinal['nama sls']

In [129]:
for index, row in slsFinal.loc[(slsFinal['ada di master sls pusat']==1) & (slsFinal['ada di master sls kabupaten']==1)].iterrows():
    idsls = row['idsls']
    try:
        slsKab = masterSLSKab[masterSLSKab['idsls']==idsls].iloc[0]
        slsPusat = masterSLSPusat[masterSLSPusat['IDSLS']==idsls].iloc[0]
        isNamaBerubah = not slsKab['nama sls']==slsPusat['NAMA SLS']
        slsFinal.loc[slsFinal['idsls']==idsls, 'nama berubah'] = 1 if isNamaBerubah else 0
        slsFinal.loc[slsFinal['idsls']==idsls, 'nama di sls kab'] = slsKab['nama sls']
    except:
        print(idsls)

In [135]:
# sls yang berubah namanya
slsBerubah = slsFinal.loc[slsFinal['nama berubah']==1]
display(slsBerubah.head())
print('banyak nama berubah :', len(slsBerubah))

,idsls,kode provinsi,provinsi,kode kabupaten,kabupaten,kode kecamatan,kecamatan,kode desa,desa,kode sls,nama sls,ada di master sls pusat,ada di master sls kabupaten,nama berubah,nama di sls kab
5,1507010016100100,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,100100,SUNGAI PUDING/BAMBU KUNING,1,1,1,RT 05 DUSUN 2
6,1507010016100200,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,100200,RIMBO SARANG ELANG,1,1,1,RT 05 DUSUN 2
16,1507010017100100,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,017,TANJUNG TAYAS,100100,SUNGAI TAPUS (001P),1,1,1,RT 008 DUSUN 3
17,1507010017100200,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,017,TANJUNG TAYAS,100200,SUNGAI SESAP (003P),1,1,1,RT 008 DUSUN 3
18,1507010017100300,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,017,TANJUNG TAYAS,100300,KEBON PROYEK (004P),1,1,1,RT 008 DUSUN 3


banyak nama berubah : 143


In [132]:
display(masterSLSKab[masterSLSKab['idsls']=='1507010016100100'])
display(masterSLSPusat[masterSLSPusat['IDSLS']=='1507010016100100'])

,provinsi,kabupaten,kecamatan,desa,nama sls,idsls,Jumlah Keluarga REGSOSEK,Nomor Urut Bangunan Terbesar REGSOSEK,"KONSENTRASI (1=KONSENTRASI, 0=NONKONSENTRASI)",Jumlah KK PEMETAAN,Jumlah KK Pertanian,PLS,PML,Koseka,kode provinsi,kode kabupaten,kode kecamatan,kode desa,kode sls
5,JAMBI,TANJUNG JABUNG BARAT,TUNGKAL ULU,BADANG,RT 05 DUSUN 2,1507010016100100,0,0,0,0,0,0,0,Toharun,15,07,010,016,100100


,KODE PROVINSI,PROVINSI,KODE KABUPATEN,KABUPATEN,KODE KECAMATAN,KECAMATAN,KODE DESA,DESA,KLASIFIKASI DESA KELURAHAN,KODE SLS,NAMA SLS,PERKIRAAN JUMLAH KK,PERKIRAAN JUMLAH KELUARGA PERTANIAN,PERKIRAAN JUMLAH KELUARGA HASIL REGSOSEK,KLASIFIKASI KONSENTRASI PERTANIAN,METODE PENDATAAN,SUMBER,IDSLS
5,15,JAMBI,07,TANJUNG JABUNG BARAT,010,TUNGKAL ULU,016,BADANG,2,100100,SUNGAI PUDING/BAMBU KUNING,0,0,0,Non Konsentrasi,Snowball,Wilkerstat,1507010016100100


In [144]:
output_folder = 'hasil'

path = output_folder+"/"+GLOBAL_VAR["output_filename"]
slsFinal.to_excel(path, index=False)

In [29]:
writer.book